In [8]:
from snap.snap import snapmerchants, cull_data
import snow.utils as sf
import pandas as pd
import editdistance

In [9]:
snapmerchants('melbourne',to_sf=False)

just work smh
Using all_data_viewer...
Established connection.
Querying load_wh.all_data.postgres
Custom data:  (136389, 1121, 'Sydney', 'Messina', 'Rosebery') 

Connection succesfully closed.
Using zans_role...
Established connection.
Querying zans_wh.zans_db.glue
Custom data:  (1248.0, 'https://www.zomato.com/sydney/boiler-house-cafe-medlow-bath\n') 

Connection succesfully closed.
Using all_data_viewer...
Established connection.
Querying load_wh.all_data.postgres
Custom data:  ('Movenpick', None, None, None, 'Melbourne', None) 

Connection succesfully closed.
Using all_data_viewer...
Established connection.
Querying load_wh.all_data.postgres
Custom data:  (1444, 'Melbourne') 

Connection succesfully closed.

Liven MELBOURNE restaurants without urls:  ['Dragon Hot Pot' 'Bubble Cup' 'MD Express' 'Spice Club' 'Aunty Franklee'
 'London K Burgers' 'San Lorenzo' 'Phancy Cafe' 'Cocorico'
 "Henry's Thai Cuisine" 'Oppa Kitchen' 'Ya Souv' 'Romanos Coffee'
 'New York Minute Burgers' 'Ten BBQ a

In [5]:
pre_snapped = sf.from_snow(schema='glue',query='select * from melbourne',to_df=False)

Using zans_role...
Established connection.
Querying zans_wh.zans_db.glue
Custom data:  (1248.0, 'https://www.zomato.com/sydney/boiler-house-cafe-medlow-bath\n') 

Connection succesfully closed.


In [6]:
pre_snap = pd.DataFrame(pre_snapped,columns=['Id','url'])

In [7]:
pre_snap

,Id,url
0,1248.0,https://www.zomato.com/sydney/boiler-house-caf...
1,435.0,None
2,1239.0,None
3,1149.0,None
4,60.0,None
5,1233.0,None
6,716.0,None
7,1230.0,None
8,1165.0,None
9,1014.0,None


In [12]:
to_snowflake = pd.concat([snap_df[['Id','url']],pre_snap])

In [1]:
import pandas as pd
import numpy as np
from snap.snap import populate_empty_branches, cull_data

In [2]:
snap_df = pd.read_csv('snapped.csv')
df,interactions = populate_empty_branches(snap_df)
snapped_df = pd.concat([snap_df,df])

Using all_data_viewer...
Established connection.
Querying load_wh.all_data.postgres
Interactions data:  ('fen_job@yahoo.com', '400958', '2019-06-13', None, None, None, Decimal('7.000000'), 'Yes', 'Sydney', 861, 'CoCo Fresh Tea & Juice', 1242, '2019-06-13', Decimal('7.000000000000')) 

Connection succesfully closed.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
interactions = interactions.rename({'merchant':'Name'},axis=1)
interactions = cull_data(interactions)
int_df = interactions[['branchId','merchantId','Name']].drop_duplicates()
int_df = int_df.rename({'branchId':'Id','merchantId':'MId','merchant':'Name'},axis=1)

In [7]:
final_df = snapped_df.merge(int_df,how='outer')
final_df = final_df.drop(['Suburb','Zomato Name','levenshtein','n'],axis=1)
final_df.url = final_df.url.replace(np.nan,'None')

In [8]:
template_df = final_df[final_df.url=='None']
template_df[['MId','Name','url']
           ].drop_duplicates(subset=['MId']
                ).to_csv('snap/manual/template_sydney.csv',index=False)

In [10]:
import os
x_df = pd.read_csv('snap/manual/sydney.csv')

In [105]:
url_dict = {}
for _, r in x_df.iterrows():
    key = int(r['MId'])
    url_dict[key] = r['url']

In [127]:
df = pd.DataFrame(zip(list(url_dict.keys())
                 ,list(url_dict.values()))
                 ,columns=['MId','url'])

In [20]:
y = x_df.merge(final_df,on='MId')

In [12]:
y[y.Name_x!=y.Name_y]

,MId,Name_x,url_x,Id,Name_y,url_y


In [29]:
snapped = final_df[final_df.url!='None']
COMPLETE = pd.concat([snapped[['Id','url']],y[['Id','url_x']].rename({'url_x':'url'},axis=1)])

In [30]:
import snow.utils as sf

In [13]:
sf.to_snow(df=to_snowflake
           ,table='sydney'
           ,schema='glue')

2019-06-19
Accepted dataframe object
Schema: glue already exists; updating sydney table.
sydney successfully updated.
Engine successfully closed
